In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tmdb-box-office-prediction/train.csv
/kaggle/input/tmdb-box-office-prediction/test.csv
/kaggle/input/tmdb-box-office-prediction/sample_submission.csv


In [7]:
train_data=pd.read_csv('../input/tmdb-box-office-prediction/train.csv', index_col='id')

In [8]:
train_data.shape

(3000, 22)

In [12]:
train_data.dropna(axis=0, subset=['revenue'], inplace=True)
y = train_data.revenue
train_data.drop(['revenue'], axis=1, inplace=True)

KeyError: ['revenue']

In [13]:
train_data.shape

(3000, 21)

In [14]:
categorical_cols = [cname for cname in train_data.columns if
                    train_data[cname].nunique() < 10 and 
                    train_data[cname].dtype == "object"]
numerical_cols = [cname for cname in train_data.columns if 
                train_data[cname].dtype in ['int64', 'float64']]

In [15]:
my_cols = categorical_cols + numerical_cols
reduced_train = train_data[my_cols].copy()

In [16]:
reduced_train.shape

(3000, 4)

In [17]:
reduced_train.head()

,status,budget,popularity,runtime
id,,,,
1,Released,14000000,6.575393,93.0
2,Released,40000000,8.248895,113.0
3,Released,3300000,64.299990,105.0
4,Released,1200000,3.174936,122.0
5,Released,0,1.148070,118.0


In [18]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [21]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


In [22]:
numerical_transformer = SimpleImputer(strategy='constant')
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [23]:
model = RandomForestRegressor(n_estimators=100, random_state=0)


In [24]:
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

In [25]:
clf.fit(reduced_train, y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='constant'),
                                                  ['budget', 'popularity',
                                                   'runtime']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['status'])])),
                ('model', RandomForestRegressor(random_state=0))])

In [26]:
test_data=pd.read_csv('../input/tmdb-box-office-prediction/test.csv',index_col='id')

In [27]:
test_data.shape

(4398, 21)

In [28]:
reduced_test=test_data[my_cols].copy()

In [29]:
reduced_test.shape

(4398, 4)

In [30]:
reduced_test.head()

,status,budget,popularity,runtime
id,,,,
3001,Released,0,3.851534,90.0
3002,Released,88000,3.559789,65.0
3003,Released,0,8.085194,100.0
3004,Released,6800000,8.596012,130.0
3005,Released,2000000,3.217680,92.0


In [31]:
preds=clf.predict(reduced_test)

In [32]:
print(preds[0:6])

[ 1854219.04  6436852.58  4967004.01 27929102.1   8410553.29 18324247.68]


In [33]:
print(len(preds))

4398


In [34]:
output = pd.DataFrame({'id': reduced_test.index,
                       'revenue': preds})
output.to_csv('sub4.csv', index=False)